In [1]:
import pdfplumber
from pdfminer.high_level import extract_text

import io
import requests
import re

import pandas as pd
import concurrent.futures

In [19]:
doc_parlement = "https://www.lachambre.be/FLWB/PDF/55/3197/55K3197001.pdf" 
doc2 = "http://www.lachambre.be/FLWB/PDF/55/3509/55K3509001.pdf"
doc3 = "http://www.lachambre.be/FLWB/PDF/55/2894/55K2894001.pdf"
doc4 = "http://www.lachambre.be/FLWB/PDF/55/2092/55K2092001.pdf"
plenary_doc = "" # 3- Séances Plénières integral - Maciej
com_integral = "" # 4- Commissions integral - Mourad
bulletin = "" # 5- Bulletins des questions et réponses écrites - Ramina
rapport = "" # 6- Rapports déposés en exécution d'une loi - Sarkis

In [2]:
def get_text(url):
    rq = requests.get(url)
    left_column_text_all_pages = ""
    right_column_text_all_pages = ""

    with pdfplumber.open(io.BytesIO(rq.content)) as pdf:
        for page in pdf.pages:
            words = page.extract_words()
            threshold_x = 300

            left_column_words = [word for word in words if word['x0'] < threshold_x]
            right_column_words = [word for word in words if word['x0'] >= threshold_x]

            left_column_text = ' '.join([word['text'] for word in left_column_words])
            right_column_text = ' '.join([word['text'] for word in right_column_words])

            left_column_text_all_pages += left_column_text + ' '
            right_column_text_all_pages += right_column_text + ' '

            left_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', left_column_text_all_pages) # remove end_line hyphens
            left_column_clean = re.sub(r'DOC 55 \d{4}/001', '', left_column_clean)
            left_column_clean = re.sub(r'\d{4}/001 DOC 55', '', left_column_clean)
            left_column_clean = re.sub(r'CHAMBRE \d+e SESSION DE LA 55e LÉGISLATURE \d{4}(?: \d+)?', '', left_column_clean)

            right_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', right_column_text_all_pages) # remove end_line hyphens
            right_column_clean = re.sub(r'DOC 55 \d{4}/001', '', right_column_clean)
            right_column_clean = re.sub(r'\d{4}/001 DOC 55', '', right_column_clean)
            right_column_clean = re.sub(r'\d{4} KAMER • \d+e ZITTING VAN DE 55e ZITTINGSPERIODE(?: \d+)?', '', right_column_clean)

    return left_column_clean, right_column_clean

    # {"fr_pdf_text" : left_column_text_all_pages, "nl_pdf_text": right_column_text_all_pages}
    

In [14]:
df = pd.read_json('apercu_complet.json')
df_pdf_not_disponible = df[df["DocumentChambre"].str.contains("pasdisponible")]
df_pdf_disponible = df[~df["DocumentChambre"].str.contains("pasdisponible")]

In [17]:
first_300 = df_pdf_disponible.iloc[:300]
first_300[['fr_text', 'nl_text']] = first_300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
first_300.to_csv("first_300.csv")

/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/1449241180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_300[['fr_text', 'nl_text']] = first_300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/1449241180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_300[['fr_text', 'nl_text']] = first_300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))


In [10]:
df_pdf_disponible[['fr_text', 'nl_text']] = df_pdf_disponible['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))


/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/3363056499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pdf_disponible[['fr_text', 'nl_text']] = df_pdf_disponible['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/3363056499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pdf_disponible[['fr_text', 'nl_text']] = df_pdf_disponible['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))


In [12]:
def process_url(url):
    left_text, right_text = get_text(url)
    return (left_text, right_text)

# Process URLs using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    results = list(executor.map(process_url, df_pdf_disponible['DocumentChambre']))

# Unpack the results and add them as new columns to the DataFrame
df_pdf_disponible[['left_column_clean', 'right_column_clean']] = pd.DataFrame(results, columns=['left', 'right'])

# Display the updated DataFrame
print(df_pdf_disponible)

                      first_title  document_number  \
0  Document parlementaire 55K3522             3522   
1  Document parlementaire 55K3520             3520   
3  Document parlementaire 55K3516             3516   
4  Document parlementaire 55K3515             3515   
5  Document parlementaire 55K3514             3514   
6  Document parlementaire 55K3513             3513   
7  Document parlementaire 55K3512             3512   
8  Document parlementaire 55K3511             3511   
9  Document parlementaire 55K3510             3510   

                                   document_page_url  \
0  https://www.lachambre.be/kvvcr/showpage.cfm?se...   
1  https://www.lachambre.be/kvvcr/showpage.cfm?se...   
3  https://www.lachambre.be/kvvcr/showpage.cfm?se...   
4  https://www.lachambre.be/kvvcr/showpage.cfm?se...   
5  https://www.lachambre.be/kvvcr/showpage.cfm?se...   
6  https://www.lachambre.be/kvvcr/showpage.cfm?se...   
7  https://www.lachambre.be/kvvcr/showpage.cfm?se...   
8  https://

/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/2240413836.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pdf_disponible[['left_column_clean', 'right_column_clean']] = pd.DataFrame(results, columns=['left', 'right'])
/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/2240413836.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pdf_disponible[['left_column_clean', 'right_column_clean']] = pd.DataFrame(results, columns=['left', 'right'])


In [13]:
df_pdf_disponible

,first_title,document_number,document_page_url,main_title,,Documentsprincipaux,DocumentChambre,Datededépôt,Typededocument,Datededistribution,...,5.SEANCEPLENIERECHAMBRE,Mots-clÕslibres,4.SEANCEPLENIERESENAT,Annexe1,Annexe2,DatededÚp˘t,PriseenconsidÚration,CompÚtence,left_column_clean,right_column_clean
0,Document parlementaire 55K3522,3522,https://www.lachambre.be/kvvcr/showpage.cfm?se...,L'évolution de la dette de l'État fédéral - Au...,,00/000-K3522/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3522/55K35...,04/08/2023,23RAPPORT,10/08/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 4 aoû...,b K elgisChe amer van volKsvertegenwoordigers...
1,Document parlementaire 55K3520,3520,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Projet de loi modifiant la loi du 7 mai 1999 p...,,00/000-K3520/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3520/55K35...,02/08/2023,02PROJETDELOI,10/08/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 2 aoû...,b K elgisChe amer van volKsvertegenwoordigers...
3,Document parlementaire 55K3516,3516,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Audit de la Direction de l'information policič...,19/07/2023,00/000-K3516/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3516/55K35...,19/07/2023,23RAPPORT,19/07/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 20 ju...,b K elgisChe amer van volKsvertegenwoordigers...
4,Document parlementaire 55K3515,3515,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Projet de loi portant diverses modifications d...,,00/000-K3515/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3515/55K35...,20/07/2023,02PROJETDELOI,03/08/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 20 ju...,b K elgisChe amer van volKsvertegenwoordigers...
5,Document parlementaire 55K3514,3514,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Projet de loi droit de la procédure pénale I.,,00/000-K3514/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3514/55K35...,20/07/2023,02PROJETDELOI,10/08/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 19 ju...,b K elgisChe amer van volKsvertegenwoordigers...
6,Document parlementaire 55K3513,3513,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Proposition de loi modifiant la loi du 26 janv...,PENDANTCHAMBRE,00/000-K3513/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3513/55K35...,19/07/2023,05PROPOSITIONDELOI,19/07/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 18 ju...,b K elgisChe amer van volKsvertegenwoordigers...
7,Document parlementaire 55K3512,3512,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Proposition de loi pour un statut démocratique...,,00/000-K3512/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3512/55K35...,NaN,05PROPOSITIONDELOI,18/07/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 18 ju...,b K elgisChe amer van volKsvertegenwoordigers...
8,Document parlementaire 55K3511,3511,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Proposition de résolution relative aux maladie...,,00/000-K3511/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3511/55K35...,18/07/2023,06PROPOSITIONDERESOLUTION,19/07/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 18 ju...,b K elgisChe amer van volKsvertegenwoordigers...
9,Document parlementaire 55K3510,3510,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Proposition de modification du Rčglement de la...,,00/000-K3510/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3510/55K35...,18/07/2023,21PROPOSITIONREGLEMENT,18/07/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# df_pdf_not_disponible["fr_text"] = "PDF not disponible"
# df_pdf_not_disponible["nl_text"] = "PDF not disponible"
df_pdf_disponible["fr_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[0])
df_pdf_disponible["nl_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[1])

In [172]:
df = pd.read_json('apercu_complet.json')

def add_text_to_df(df):
    df_pdf_not_disponible = df[df["DocumentChambre"].str.contains("pasdisponible")]
    df_pdf_disponible = df[~df["DocumentChambre"].str.contains("pasdisponible")]
    df_pdf_disponible["fr_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[0])
    df_pdf_disponible["nl_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[1])
    df_pdf_not_disponible["fr_text"] = "PDF not disponible"
    df_pdf_not_disponible["nl_text"] = "PDF not disponible"

In [ ]:
# def new_get_text(df):
#     try:
#         rq = requests.get(df["DocumentChambre"])
#         left_column_text_all_pages = ""
#         right_column_text_all_pages = ""

#         with pdfplumber.open(io.BytesIO(rq.content)) as pdf:
#             for page in pdf.pages:
#                 words = page.extract_words()
#                 threshold_x = 300

#                 left_column_words = [word for word in words if word['x0'] < threshold_x]
#                 right_column_words = [word for word in words if word['x0'] >= threshold_x]

#                 left_column_text = ' '.join([word['text'] for word in left_column_words])
#                 right_column_text = ' '.join([word['text'] for word in right_column_words])

#                 left_column_text_all_pages += left_column_text + ' '
#                 right_column_text_all_pages += right_column_text + ' '

#                 left_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', left_column_text_all_pages) # remove end_line hyphens
#                 left_column_clean = re.sub(r'DOC 55 \d{4}/001', '', left_column_clean)
#                 left_column_clean = re.sub(r'\d{4}/001 DOC 55', '', left_column_clean)
#                 left_column_clean = re.sub(r'CHAMBRE \d+e SESSION DE LA 55e LÉGISLATURE \d{4}(?: \d+)?', '', left_column_clean)

#                 right_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', right_column_text_all_pages) # remove end_line hyphens
#                 right_column_clean = re.sub(r'DOC 55 \d{4}/001', '', right_column_clean)
#                 right_column_clean = re.sub(r'\d{4}/001 DOC 55', '', right_column_clean)
#                 right_column_clean = re.sub(r'\d{4} KAMER • \d+e ZITTING VAN DE 55e ZITTINGSPERIODE(?: \d+)?', '', right_column_clean)

#         df["fr_text"] = left_column_clean
#         df["nl_text"] = right_column_clean
#         return df

#     except Exception as e:
#         print(type(e))
#         return e

# for i in df_pdf_disponible.loc(): 
#     print(i)
#     # new_get_text(i)

# # search_links = get_search_url_list(end_page=334)

# # urls = list(itertools.chain.from_iterable(thread_map(get_urls_from_search_page, search_links)))

# # with requests.Session() as session:
#     # new_df = df for df in thread_map(partial(new_get_text, session=session), df_pdf_disponible)

# # properties.to_csv("properties.csv")

In [ ]:
df_pdf_disponible